In [1]:
import sys
from pathlib import Path

# Add project root to Python path
project_root = str(Path.cwd().parent)  # Assumes notebook is in integration/ directory
if project_root not in sys.path:
    sys.path.append(project_root)

import json
from typing import List
from src.auto_chart_patterns.chart_pattern import ChartPattern
from src.auto_chart_patterns.trendline_patterns import find_trend_lines
from src.auto_chart_patterns.reversal_patterns import find_reversal_patterns
from src.auto_chart_patterns.rsi_div_patterns import find_rsi_divergences
from src.auto_chart_patterns.flag_pennant_patterns import find_flags_and_pennants
from tests.integration.helpers import get_data_for_ticker, PATTERN_CATEGORIES, START_DATE, END_DATE

ticker_list = ["BTCUSDT", "SOLUSDT", "ETHUSDT", "AVAXUSDT", "DOGEUSDT", "SHIBUSDT"]
def build_test_cases(ticker: str, category: int, find_patterns_func, zigzag, default_properties):
    prices = get_data_for_ticker(ticker, start_date=START_DATE,
                                 end_date=END_DATE)
    patterns: List[ChartPattern] = []
    if category == 4:
        find_patterns_func(zigzag.backcandles, zigzag.forwardcandles,
                          default_properties, patterns, prices)
    else:
        zigzag.calculate(prices)
        for i in range(0, len(zigzag.zigzag_pivots)):
            find_patterns_func(zigzag, i, default_properties, patterns)

    with open(f'../tests/integration/fixtures/{PATTERN_CATEGORIES[category]}/'
              f'{ticker}_{PATTERN_CATEGORIES[category]}_patterns.json', 'w') as f:
        # format json
        json.dump([pattern.dict() for pattern in patterns], f, indent=4)


## Flags & pennants

In [2]:
from tests.integration.helpers import zigzag_fp, scan_properties_fp

for ticker in ticker_list:
    build_test_cases(ticker, 1, find_flags_and_pennants,
                    zigzag_fp, scan_properties_fp)


## Trend lines

In [3]:
from tests.integration.helpers import zigzag_tl, scan_properties_tl

for ticker in ticker_list:
    build_test_cases(ticker, 2, find_trend_lines,
                    zigzag_tl, scan_properties_tl)


## Reversal patterns

In [4]:
from tests.integration.helpers import zigzag_rp, scan_properties_rp

for ticker in ticker_list:
    build_test_cases(ticker, 3, find_reversal_patterns,
                    zigzag_rp, scan_properties_rp)


## RSI divergences

In [2]:
from tests.integration.helpers import zigzag_rsi, scan_properties_rsi

for ticker in ticker_list:
    build_test_cases(ticker, 4, find_rsi_divergences,
                    zigzag_rsi, scan_properties_rsi)
